### https://zhuanlan.zhihu.com/p/176241946

In [1]:
import datetime
import backtrader as bt
import backtrader.indicators as bi
# import backtest
import pandas as pd
import numpy as np
# import math
# import matplotlib.pyplot as plt

In [2]:
from lutils.stock import LTdxHq

In [3]:
class GridStrategy(bt.Strategy):
    params = (
        ("printlog", True),
        ("top", 12),
        ("buttom", 7),
    )
    
    def __init__(self):
        self.mid = (self.p.top + self.p.buttom)/2.0
        # 百分比区间计算
        #这里多1/2，是因为arange函数是左闭右开区间。
        perc_level = [x for x in np.arange(1 + 0.02 * 5, 1 - 0.02 * 5 - 0.02/2, -0.02)]
        # 价格区间
        # print(self.mid)
        self.price_levels = [self.mid * x for x in perc_level]
        # 记录上一次穿越的网格
        self.last_price_index = None
        # 总手续费
        self.comm = 0.0
 
    def next(self):
        # print(self.last_price_index)
        # 开仓
        if self.last_price_index == None:
            # print("b", len(self.price_levels))
            for i in range(len(self.price_levels)):
                price = self.data.close[0]
                # print("c", i, price, self.price_levels[i][0])
                if self.data.close[0] > self.price_levels[i]:
                    self.last_price_index = i
                    self.order_target_percent(target=i/(len(self.price_levels) - 1))
                    print("a")
                    return
        # 调仓
        else:
            signal = False
            while True:
                upper = None
                lower = None
                if self.last_price_index > 0:
                    upper = self.price_levels[self.last_price_index - 1]
                if self.last_price_index < len(self.price_levels) - 1:
                    lower = self.price_levels[self.last_price_index + 1]
                # 还不是最轻仓，继续涨，再卖一档
                if upper != None and self.data.close > upper:
                    self.last_price_index = self.last_price_index - 1
                    signal = True
                    continue
                # 还不是最重仓，继续跌，再买一档
                if lower != None and self.data.close < lower:
                    self.last_price_index = self.last_price_index + 1
                    signal = True
                    continue
                break
            if signal:
                self.long_short = None
                self.order_target_percent(target=self.last_price_index/(len(self.price_levels) - 1))
 
    # 输出交易记录
    def log(self, txt, dt = None, doprint = False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))
 
    def notify_order(self, order):
        # 有交易提交/被接受，啥也不做
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 交易完成，报告结果
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('执行买入, 价格: %.2f, 成本: %.2f, 手续费 %.2f' % (order.executed.price, order.executed.value, order.executed.comm))
                self.buyprice = order.executed.price
                self.comm += order.executed.comm
            else:
                self.log('执行卖出, 价格: %.2f, 成本: %.2f, 手续费 %.2f' % (order.executed.price, order.executed.value, order.executed.comm))
                self.comm += order.executed.comm
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log("交易失败")
        self.order = None
 
    # 输出手续费
    def stop(self):
        self.log("手续费:%.2f 成本比例:%.5f" % (self.comm, self.comm/self.broker.getvalue()))



In [4]:
lt = LTdxHq()

In [5]:
code = '603636' # 510300 000032 300142 603636 600519 688567

In [6]:
df = lt.get_k_data_daily(code, qfq=True)

In [7]:
# df = df['2021-01-01':]

In [8]:
top = df.close.min() * .8
buttom = df.close.max() * 1.1

In [9]:
top

3.5840000000000005

In [10]:
buttom

27.511000000000003

In [11]:
df.index = pd.to_datetime(df.index)

In [12]:
cerebro = bt.Cerebro(oldtrades=True)

feed = bt.feeds.PandasData(dataname=df, openinterest=None, compression=1, timeframe=bt.TimeFrame.Minutes)
cerebro.adddata(feed)

# top = 4.2
# buttom = 3.5
cerebro.addstrategy(GridStrategy, top=top, buttom=buttom)

# 小场面1万起始资金
cerebro.broker.setcash(10000.0)

# 手续费万5
cerebro.broker.setcommission(0.0005)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

result = cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
a
2015-04-27, 执行买入, 价格: 14.35, 成本: 8925.70, 手续费 4.46
2015-04-29, 执行买入, 价格: 13.31, 成本: 1064.80, 手续费 0.53
2015-05-12, 执行卖出, 价格: 16.41, 成本: 4981.02, 手续费 2.87
2015-05-13, 执行卖出, 价格: 15.50, 成本: 967.74, 手续费 0.53
2015-05-15, 执行卖出, 价格: 16.22, 成本: 2063.56, 手续费 1.18
2015-05-19, 执行卖出, 价格: 17.40, 成本: 1978.18, 手续费 1.21
2015-06-17, 执行买入, 价格: 15.91, 成本: 4661.63, 手续费 2.33
2015-06-18, 执行卖出, 价格: 16.47, 成本: 1240.98, 手续费 0.64
2015-06-19, 执行买入, 价格: 15.33, 成本: 1226.40, 手续费 0.61
2015-06-23, 执行买入, 价格: 14.16, 成本: 5890.56, 手续费 2.95
2015-06-24, 执行卖出, 价格: 14.85, 成本: 1230.13, 手续费 0.62
2015-06-26, 执行买入, 价格: 13.58, 成本: 2240.70, 手续费 1.12
2015-11-09, 执行卖出, 价格: 14.19, 成本: 1092.20, 手续费 0.53
2015-11-10, 执行卖出, 价格: 15.93, 成本: 5854.19, 手续费 3.20
2015-11-11, 执行卖出, 价格: 16.09, 成本: 1208.70, 手续费 0.67
2015-11-12, 执行卖出, 价格: 17.74, 成本: 3393.10, 手续费 2.07
2015-11-16, 执行买入, 价格: 14.95, 成本: 6234.15, 手续费 3.12
2015-11-17, 执行卖出, 价格: 17.39, 成本: 5008.25, 手续费 2.91
2015-11-18, 执行卖出, 价格: 17.04, 成本: 1225.90, 手续费 

In [13]:
cerebro.plot(
    iplot=False,
    start=datetime.date(2020, 1, 1),
    end=datetime.date(2021, 9, 30),
    style='candlestick',
    barup='red',
    bardown='green',
)

[[<Figure size 1189x553 with 4 Axes>]]

In [8]:
# start = "2018-01-01"
# end = "2021-07-05"
# name = ["300etf"]
# code = ["510300"]
# backtest = backtest.BackTest(GridStrategy, start, end, code, name, 100000)
# result = backtest.run()
# # backtest.output()
# print(result)